## **Case study: Predict stent deployed configuration in real time**

In this class, we will build a data-driven reduced order model (ROM) of stent deployment capable of predicting in **real time** the stent deployed configuration given a certain vessel geometry.

### **What are stents?**
Stents are small, tube-like devices used to hold open narrowed or blocked arteries or other blood vessels in the body. They are often made of metal or plastic mesh and are commonly used in procedures like angioplasty to restore blood flow in cases of coronary artery disease. Some stents are coated with medication to prevent the artery from narrowing again, while others are simple mechanical supports.

There exist two main types of stents. **Balloon-expandable stents** are made from materials like stainless steel and are deployed by inflating a balloon inside the stent, which forces it to expand. They provide high radial force and are ideal for precise placement, such as in coronary arteries. In contrast, **self-expandable stents** are made from shape-memory materials like nitinol. They expand on their own after being released from a catheter and are more flexible, making them suitable for areas that experience movement or compression, like peripheral arteries.

In this class, we will focus on the self-expandable stents.

![Balloon-expandable stent](https://i.imgur.com/dWsdVgo.jpeg)
![Self-expandable stent](https://i.imgur.com/3d7OVob.jpeg)

### **Why we need a ROM of stents?**
Examples of why a ROM for stent deployment is needed:
*  **Optimization Study:** A ROM enables running multiple simulations with varying geometrical or material properties of the stent to find the combination that maximizes specific performance metrics, such as radial force, flexibility, and fatigue resistance.
*   **Planning Software:** By using a ROM, clinicians can compare different treatment options before actual deployment, evaluating various devices, sizes, and deployment sites to choose the most suitable option for each patient.
*   **Intraoperative Support:** ROM can help visualize the deployed configuration overlaid on fluoroscopy images in real-time during the intervention, assisting surgeons with precise positioning and adjustments.

### **What is the focus of this class?**
In this class, we will focus on building a ROM of the deployed configuration of a stent given a set of parameters describing the vessel geometry. As this is an introductory class, we will work on an idealised vessel geometry. Our parameters will represent the curvature and radius of this vessel as long as the clinical decision on where to position the stent.

![Parametric vessel](https://i.imgur.com/tccLgJy.png)

Eventually, this model could be translated to a realistic case with patient-specific geometries using a geometrical parameters or statistical parameters, computed by building a statistical shape model (see class 18/12/24).

![Patient-specific vessel](https://i.imgur.com/dql3B00.png)



### **How is the class structured?**
1. Mathematical definition of the problem
2. Sampling
3. Building the database
4. Building and testing the ROM

![Schema](https://i.imgur.com/cjbySFE.png)

### **Reference**
About stents:

*   https://www.sciencedirect.com/science/article/pii/S1350453310002420
*   https://www.sciencedirect.com/science/article/pii/S0021929012003454
*   https://www.sciencedirect.com/science/article/pii/S0021929012001789via%3Dihub

Review on stents:
*   https://www.sciencedirect.com/science/article/pii/S2666522023000175
*   https://onlinelibrary.wiley.com/doi/full/10.1002/cnm.3529

About reduced order modelling:

* https://hal.science/hal-01007232/document
* https://amses-journal.springeropen.com/articles/10.1186/s40323-015-0049-1
* https://www.degruyter.com/document/doi/10.1515/9783110499001-008/html


About this class:
* https://www.frontiersin.org/journals/physiology/articles/10.3389/fphys.2023.1148540/full
* https://www.sciencedirect.com/science/article/pii/S0045782518303487?casa_token=m-QUUFRG63IAAAAA:bDQR4oqhKto5bPIMnnyrbkdYk88o61SO0xvOC6Rc_8P8O6cvFljDK479s6fRatoobbGr2B2msA




## **Setting up the environment**


In [ ]:
# Setting up the environment
import os
os.chdir('/content')

import shutil
import os

# Check if the folder 'NotebookROM' exists before attempting to delete it to reinstall it
if os.path.exists('NotebookROM'):
    # Remove the folder and all its contents
    shutil.rmtree('NotebookROM')

# Pull the repo with data and install packages for the notebook if we're in Google Colab
import os
try:
  import google.colab # Check if we're in Google Colab
  !sudo apt install libgl1-mesa-glx xvfb
  !pip install pyvista[jupyter] -qq
  current_directory = os.path.basename(os.getcwd())
  # If we're not in the right directory or the directory doesn't exist, clone the repo
  if not (os.path.isdir('NotebookROM') or current_directory == 'NotebookROM'):
    !git clone https://github.com/bisighinibeatrice/NotebookROM.git
  if os.path.isdir('NotebookROM'):
    %cd NotebookROM
except:
    pass

In [ ]:
# Import the functions
from importlib import reload
import utils
utils = reload(utils) # Reload the module in case it has changed
import numpy as np
from ipywidgets import interactive

## **1. Mathematical and physical definition of the problem**

The problem we are modelling consists of two entities: the stent and the vessel. A visualisation tool (`Visualizer`) has been implemented to visualize stents and vessels. Here are the documentation and examples of how to use it.

In [ ]:
help(utils.Visualizer)

The **stent** we are modeling is a braided design, consisting of 24 interlaced wires with a relaxed radius of 2.7 mm and a length of 12 mm. It is represented using **beam elements**. The mesh data, including node positions and connectivity, is saved in the `Input` folder.

> You can visualize the stent using the function `add_stent_from_file()` provided by the `Visualizer` object by passing in input the position and connectivity files.

In [ ]:
# Create a visualizer object and add a stent to it
viz = utils.Visualizer()
viz.add_stent_from_file("Input/pos_stent.txt", "Input/conn_stent.txt")
viz.show()

**EXERCISE**: How many nodes compose the stent? How many elements?

The **vessel** geometry is represented through a parameterized model, where the parameters serve as inputs for our ROM. The vessel's shape is defined by its **radius** and its **centerline**. With fixed starting and ending points, the centerline is parameterized using a third point, referred to as the control point, which is adjusted within the x-z plane.

To create a smooth curve that connects these points, we employ Bernstein basis polynomials to weight the contributions of the three points —start point, control point, and endpoint. This approach effectively blends the points, resulting in a continuous and visually appealing path. This technique is widely used in computer graphics and computational geometry for generating smooth paths and shapes. The `spline_interpolation()` function of the the `Visualizer` object implements this process, computing an interpolated point along a cubic Bézier curve defined by the three points.

An additional parameter is considered in our analysis: the **deploy site** of the stent, which indicates its position at the beginning of deployment. This parameter ranges from 0 to 1, where 0 corresponds to the stent being positioned at the start of the vessel (proximal position) and 1 indicates that the stent is located at the end of the vessel (distal position).

![Parametric vessel](https://i.imgur.com/dx4kMDu.png)

> You can visualize a generic vessel using the functions provided by the `Visualizer` object. The centerline is shown in black, the control point in red, and the deploy site in green. The default values provide a straight vessel.

In [ ]:
# Create a visualizer object and add a vessel to it
viz = utils.Visualizer()
viz.add_vessel()
viz.show()

By changing the input values, we can update the shape of the vessel thought the function `update_vessel()`.

In [ ]:
# Update the control point and radius
control_point = np.array([60, 0, 25], dtype=np.float64)
vessel_radius = 2
deploy_site = 0.5

viz.update_vessel(control_point, vessel_radius, deploy_site)
viz.show()

Through an interactive tool, we can play with the shape of the vessel by changing the parameters value using 4 sliders.

In [ ]:
# Interactively play with the parameters of the vessel
viz = utils.Visualizer()
viz.add_vessel()

def update_vessel(x, z, r, s):
    control_point = np.array([x, 0, z], dtype=np.float64)
    viz.update_vessel(control_point, r, s)
    viz.show()

w = interactive(update_vessel, x=(-50, 50, 1), z=(0, 50, 1), r=(1, 20, 1), s=(0, 1, 0.1))
for c in w.children:
    c.continuous_update = False
display(w)

## **2. Sampling**

In reduced order modelling, the sampling step is crucial for generating a representative set of data points that capture the variability of the system.The Latin Hypercube Sampling (LHS) technique is commonly used for this purpose. LHS is a statistical method that divides the parameter space into equally probable intervals and ensures that samples are distributed uniformly across the entire space. This technique reduces the risk of clustering and improves the efficiency of sampling compared to simple random sampling, making it ideal for high-dimensional problems. It helps in covering the parameter space more thoroughly with fewer samples, which enhances the accuracy of the ROM while minimizing computational costs.

This is a Python example of the use of LHS for sampling two parameters (`num_dimensions`). Plot the distribution of the points for different number of samples (`num_samples`).

In [ ]:
import numpy as np
from scipy.stats import qmc
import matplotlib.pyplot as plt

# Define number of parameters
num_dimensions = 2

# Define sample size
num_samples = 100

# Create a Latin Hypercube Sampler
lhs_sampler = qmc.LatinHypercube(d=num_dimensions)
lhs_sample = lhs_sampler.random(n=num_samples)

# Plot the results
plt.scatter(lhs_sample[:, 0], lhs_sample[:, 1], c='red', label='Samples')
plt.title('2D Latin Hypercube Sampling')
plt.xlabel('Parameter 1 (Range: -5 to 5)')
plt.ylabel('Parameter 2 (Range: -4 to 8)')
plt.grid(True)
plt.legend()
plt.show()

The LHS provides values between 0 and 1. How can we change the ranges in which the parameters are varying? We need to rescale the data.

To scale the data from the unit interval [0,1] to a specified range [a,b], we can use this equation:

$x_{\text{scaled}} = a + (b - a) \cdot x_{\text{sample}}$

where $x_{\text{scaled}}$ is the scaled value, $x_{\text{sample}}$ is the original LHS sample in the range [0,1], $a$ is the lower bound of the target range, and $b$ is the upper bound of the target range.

Implement this equation to rescale the data and plot the scaled results.

In [ ]:
# Parameters bounds
param1_bounds = (-5.0, 5.0)  # Range for Parameter 1
param2_bounds = (-4.0, 8.0)  # Range for Parameter 2

# Updated bounds for the two parameters
bounds = [param1_bounds, param2_bounds]

# Scale the LHS sample to the given bounds
scaled_lhs_sample = np.empty_like(lhs_sample)
for i, (lower, upper) in enumerate(bounds):
    scaled_lhs_sample[:, i] = lower + lhs_sample[:, i] * (upper - lower)

# Plot the results
plt.scatter(scaled_lhs_sample[:, 0], scaled_lhs_sample[:, 1], c='red', label='Samples')
plt.title('2D Latin Hypercube Sampling')
plt.xlabel('Parameter 1 (Range: -5 to 5)')
plt.ylabel('Parameter 2 (Range: -4 to 8)')
plt.grid(True)
plt.legend()
plt.show()

Let's go back to our original problem of interest. We have 4 parameters (vessel radius, control point x and z coordinates, and deploy site), which are varying in these ranges:
*   `vessel_radius_bounds = (1.0, 2.5)`
*   `control_point_x_bounds = (0.0, 50.0)`
*   `control_point_z_bounds = (0.0, 50.0)`
*   `deploy_site_bounds = (0.01, 1.0)`

**EXERCISE:**  Change the LHS code to compute `num_samples = 5` with LHS (considering the ranges indicated above) and visualise them.

## **3. Building the database**

To create the ROM, we considered four scenarios with an increasing number of cases: 100, 200, 500, and 1000. Using the LHS method, we determined the parameter values and generated the corresponding geometries using a dedicated Python routine. For each of these geometries, we then ran a simulation of stent deployment using an in-house, finite-element software. This simulation consists in computing, starting from the stent configuration previously shown and the deploy site defined by the LHS, the deployed configuration of the stent.

These simulations are performed in three steps:

1. First the stent is crimped to fit within the vessel. ![Crimping](https://i.imgur.com/xSBHAJf.gif)

2. Then, positioned along the centerline of the vessel. ![Positioning](https://i.imgur.com/ChGbkJU.gif)

3. And finally, deployed. ![Deployment](https://i.imgur.com/hsWHPZJ.gif)

As this topic will not be covered in the current class, additional information on finite-element simulations of stent deployment can be found in the literature provided as reference material.

For the following step, the data corresponding to the input (`parameters_$N.txt`) and output (`simulations_$N.txt`) of these simulations was saved in the `Input` folder. The output consists in the displacements of the stent nodes at the end of the simulation. We can visualise a sample case using the `Visualizer` tool.

In [ ]:
# Specify the database size and the case
N = 100
line_number = 1  # Line number to be read from the displacements file

# Specify the names of the input files
position_file = "Input/pos_stent.txt"  # Position data file (positions for the stent)
connectivity_file = "Input/conn_stent.txt"  # Connectivity data file
displacement_file = f"Input/simulations_{N}.txt"  # Displacement data file (displacement vector for simulations)

# Read the position matrix from position_file
position = np.loadtxt(position_file)
connectivity = np.loadtxt(connectivity_file).astype(np.int32)

# Open displacement_file to read the specified line (line_number)
with open(displacement_file, 'r') as displacement_file_handle:
    # Read all lines into a list
    lines = displacement_file_handle.readlines()
    if line_number <= len(lines):  # Ensure that line_number is within the number of available lines
        displacement_vector = list(map(float, lines[line_number - 1].strip().split()))  # Read and convert the line to floats
    else:
        raise ValueError(f"Line {line_number} does not exist in the file.")

# Reshape the displacement vector into a matrix with 3 columns
displacement = np.array(displacement_vector).reshape(-1, 3)

# Add the position matrix and displacement matrix element-wise to get the final position matrix
final_position = position + displacement

# Create a visualizer object and add a stent to it
visualizer = utils.Visualizer()
visualizer.add_stent(final_position, connectivity)

# Update the control point and radius based on parameters from a different file
parameter_file = f"Input/parameters_{N}.txt"

with open(parameter_file, 'r') as parameter_file_handle:
    # Read all lines from the parameter file
    lines = parameter_file_handle.readlines()
    if line_number <= len(lines):  # Ensure that line_number is within the number of available lines
        parameter_vector = list(map(float, lines[line_number - 1].strip().split()))  # Read and convert the line to floats
    else:
        raise ValueError(f"Line {line_number} does not exist in the file.")

# Define the control point, vessel radius, and deployment site from the parameters
control_point = np.array([parameter_vector[1], 0, parameter_vector[2]], dtype=np.float64)
vessel_radius = parameter_vector[0]
deployment_site = parameter_vector[3]

# Update the vessel display with the new control point and radius
visualizer.add_vessel(control_point, vessel_radius, deployment_site)
visualizer.show()

## **4. Building and testing the ROM**

Machine learning is a key technique used in building ROMs for complex simulations. In this class, we will focus on regression models. **Regression** involves fitting a mathematical function to approximate the relationship between input parameters (e.g., geometry, material properties) and output variables (e.g., stress, displacement). There are many machine learning techniques, ranging from simpler models like linear regression to more complex models capable of capturing nonlinear relationships, such as neural networks.

In this class, we will use Gaussian process regression (GPR), as it is well-suited for small training datasets and has previously been applied to nonlinear structural problems.

In [ ]:
# Import Libraries
import numpy as np
from sklearn.decomposition import PCA
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.gaussian_process.kernels import ConstantKernel as C
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import time

# Load parameters and simulations from text files
def load_data(parameters_file, simulations_file):
    parameters = np.loadtxt(parameters_file)
    simulations = np.loadtxt(simulations_file)
    return parameters, simulations

Initially, we will start from the smallest database (number of cases: `N = 100`) and use the data in its raw form as input for the machine learning model. The data has to be split between training set (used to train the model) and testing set (used to test the model, not used during the training).

In [ ]:
# Load the data
N = 100
parameters_file = f"Input/parameters_{N}.txt"  # Replace with your actual file name
simulations_file = f"Input/simulations_{N}.txt"  # Replace with your actual file name
parameters, simulations = load_data(parameters_file, simulations_file)
indices = np.arange(len(parameters))

# Split the data into training and test sets
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(parameters, simulations, indices, test_size=0.25, random_state=42)

In Gaussian Process Regression, the choice of kernel function defines the behavior of the model. It defines how similar points influence each other in predictions. Here, we’ll use a common choice: a combination of the RBF kernel and a constant kernel.

In [ ]:
# Build the Kriging model on the data: define a kernel for the Gaussian Process
initial_kernel = C(1.0, (1e0, 1e3)) * RBF(1.0, (1e-2, 1e2))

Then, we can create the GPR model and fit the data.

In [ ]:
# Create the GaussianProcessRegressor with the defined kernel
gp = GaussianProcessRegressor(kernel=initial_kernel, n_restarts_optimizer=10, random_state=42)

# Fit the Gaussian Process model to the reduced data
gp.fit(X_train, y_train)

Now that it has been trained, we can use it to make prediction.

In [ ]:
# Predict using the Kriging model on the test data
y_pred = gp.predict(X_test)

The model has been successfully trained and now we need to **evaluate** its performance. For a regression model, we can assess its performance using metrics such as:
*   **Mean absolute error** (**MAE**): Measures the average magnitude of errors in absolute terms (**Pros:** Intuitive to understand and interpret. **Cons:** Does not penalize large errors heavily).
*   **Mean square error** (**MSE**): Measures the average squared difference between actual and predicted values (**Pros:** Penalizes larger errors more than MAE. **Cons:**  Sensitive to outliers).
*   **Root Mean Squared Error** (**RMSE**): The square root of MSE, expressed in the same units as the original data (**Pros:** Easy to interpret due to its unit consistency. **Cons:** Like MSE, it is sensitive to outliers).

The expressions for the error metrics are:

*   **Mean Squared Error (MSE):** $\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$

*    **Mean Absolute Error (MAE):** $\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$

*    **Root Mean Squared Error (RMSE):** $\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$

with:

*    $y_i$: The true value or the actual value from the dataset.
*    $\hat{y}_i$: The predicted value for the radius obtained from the reduced-order model.
*   $n$: The total number of data points or samples in the dataset.


In [ ]:
# Calculate MSE, MAE, and RMSE for each case, including standard deviation
mae_vec = []
rmse_vec = []
mse_vec = []

for i in range(y_test.shape[0]):
    mae = np.mean(np.abs(y_test[i, :] - y_pred[i, :]))
    mse = np.mean((y_test[i, :] - y_pred[i, :]) ** 2)
    rmse = np.sqrt(mse)

    mae_vec.append(mae)
    mse_vec.append(mse)
    rmse_vec.append(rmse)

# Calculate the mean and standard deviation for each error metric
mean_mae = np.mean(mae_vec)
std_mae = np.std(mae_vec)

mean_mse = np.mean(mse_vec)
std_mse = np.std(mse_vec)

mean_rmse = np.mean(rmse_vec)
std_rmse = np.std(rmse_vec)

# Print the results
print(f"MAE: {mean_mae} mm (Standard Deviation: {std_mae} mm)")
print(f"MSE: {mean_mse} (Standard Deviation: {std_mse})")
print(f"RMSE: {mean_rmse} mm (Standard Deviation: {std_rmse} mm)")

Absolute errors, expressed in millimeters, are straightforward to interpret. However, their significance becomes even clearer when contextualized with a relative error. A relative error, expressed as a percentage, relates the absolute error (e.g., RMSE) to a physical reference, such as the average radius of the vessels in our database. This approach provides a normalized measure of error, allowing for better understanding and comparison across different scales.

Below is the code to compute the average radius from a file where the first element of each row represents the radius of a vessel. This average radius will serve as the reference for calculating the relative error:

In [ ]:
# Specify the file path
parameter_file = f"Input/parameters_{N}.txt"

# Initialize a list to store radii
radii = []

# Open and read the file
with open(parameter_file, 'r') as file:
    for line in file:
        # Split the line into elements and take the first one as the radius
        radius = float(line.split()[0])  # Assumes the file is space-separated
        radii.append(radius)

# Compute the average radius
average_radius = sum(radii) / len(radii) if radii else 0

print(f"Average Radius: {average_radius}")

With the average_radius value calculated, you can now compute the relative error as:

$\text{Relative Error (%)} = \left( \frac{\text{RMSE}}{\text{Average Radius}} \right) \times 100$


In [ ]:
# Compute the relative error based on the mean radius
relative_error = (mean_rmse / average_radius) * 100
print(f"Relative Error: {relative_error:.2f}%")

The errors are quite high, which means that the machine learning model is struggling to establish a mapping between the input and output when using the raw data.

This could be due to the large variation between the input and output data. To evaluate this, we can compute statistical measures like the variance or standard deviation for both the input and output datasets. These metrics will help us quantify the spread or dispersion in the data, providing insight into whether significant differences in scale or range between the input and output are affecting the model's ability to learn.

In [ ]:
# Calculate standard deviation for input and output
input_std_dev = np.std(parameters, axis=0)
output_std_dev = np.std(simulations, axis=0)

print(input_std_dev)
print(output_std_dev)

To improve performance, we can **standardize** the data, which involves transforming the features to have a mean of zero and a standard deviation of one.

In [ ]:
# Standardize both training and test sets for the output
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# Standardize both training and test sets for the input
scaler_x = StandardScaler()
X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.transform(X_test)

Let's visualise the change in variance.

In [ ]:
# Compute the mean values for the first 10 output variables before and after scaling
mean_before = np.mean(y_train[:, :10], axis=0)
mean_after = np.mean(y_train_scaled[:, :10], axis=0)

# Compute the standard deviations for the first 10 output variables before and after scaling
std_before = np.std(y_train[:, :10], axis=0)
std_after = np.std(y_train_scaled[:, :10], axis=0)

# Plot the means with error bars representing standard deviation before scaling
plt.figure(figsize=(10, 6))
plt.errorbar(range(1, 11), mean_before, yerr=std_before, fmt='o', label='Before Scaling', color='blue', capsize=5)
plt.title('Mean and Standard Deviation for the First 10 Output Variables (Before Scaling)')
plt.xlabel('Output Variable')
plt.ylabel('Mean Value')
plt.xticks(range(1, 11))
plt.legend()
plt.grid(True)
plt.show()

# Plot the means with error bars representing standard deviation after scaling
plt.figure(figsize=(10, 6))
plt.errorbar(range(1, 11), mean_after, yerr=std_after, fmt='o', label='After Scaling', color='orange', capsize=5)
plt.title('Mean and Standard Deviation for the First 10 Output Variables (After Scaling)')
plt.xlabel('Output Variable')
plt.ylabel('Mean Value')
plt.xticks(range(1, 11))
plt.legend()
plt.grid(True)
plt.show()

**EXERCIZE:**  Train again the machine learning model but with the scaled data now.

In [ ]:
# Build the Kriging model on the data: define a kernel for the Gaussian Process
initial_kernel = C(1.0, (1e-1, 1e3)) * RBF(1.0, (1e-2, 1e2))

# Create the GaussianProcessRegressor with the defined kernel
gp = GaussianProcessRegressor(kernel=initial_kernel, n_restarts_optimizer=10, random_state=42)

# Fit the model
gp.fit(, )

# Predict using the Kriging model on the test data
y_pred_scaled = gp.predict()

# Inverse transform to reconstruct the original output
y_pred = scaler_y.inverse_transform()

# Calculate RMSE for each case
rmse_vec = []
for i in range(y_test.shape[0]):
    rmse = np.sqrt(mean_squared_error(y_test[i, :], y_pred[i, :]))
    rmse_vec.append(rmse)

print(f"RMSE: {np.mean(rmse_vec)} mm")

# Compute the relative error based on the mean radius
relative_error = (np.mean(rmse_vec) / average_radius) * 100
print(f"Relative Error: {relative_error:.2f}%")

The accuracy of the machine learning model is much better! But the code still takes a lot of time to run. Let's quantify this time.

In [ ]:
# Measure the time taken to fit the model
start_time = time.time()
gp.fit(X_train_scaled, y_train_scaled)
end_time = time.time()

# Compute the computational time
fit_time = end_time - start_time
print(f"Time taken to fit the Gaussian Process model: {fit_time:.4f} seconds")

This long training time may be due to the complexity of learning all of these outputs simultaneously.

**EXERCISE:** How many output is our system composed of?

To reduce the training time, we can **compress** this output, and one effective approach is to use the Principal Component Analysis (PCA). This technique reduces the dimensionality of the data by transforming it into a set of uncorrelated variables, known as principal components, which capture the most significant variance in the output while minimizing information loss.

In [ ]:
# Create the PCA on the output (simulations)
n_components = 25
pca_simulations = PCA(n_components=n_components)

# Fit and transform the output
y_train_scaled_reduced = pca_simulations.fit_transform(y_train_scaled)

# Plot the cumulative explained variance
explained_variance = np.cumsum(pca_simulations.explained_variance_ratio_)

# Create the plot
plt.figure(figsize=(8, 6))
plt.plot(range(1, n_components + 1), explained_variance, marker='o', linestyle='--', color='b')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Convergence: Cumulative Explained Variance vs. Number of Components')
plt.grid(True)
plt.show()

This plot shows the cumulative explained variance against the number of principal components in a PCA analysis:

* **X-axis**: Number of principal components added.
* **Y-axis**: Cumulative explained variance—how much of the data’s variance is captured as more components are included.
* **Curve**: The curve rises quickly at first, indicating that the initial components capture most of the variance, then levels off, suggesting diminishing returns from additional components.

This plot helps identify the "elbow point," where adding more components provides minimal additional explained variance, guiding you to select an optimal number of components for dimensionality reduction.

This plot helps identify the "elbow point", where adding more components provides minimal additional explained variance, guiding you to select an optimal number of components for dimensionality reduction.

To compute the "elbow point" for PCA, which represents the optimal number of components beyond which additional components contribute less to explaining the variance, we can analyze the cumulative explained variance and look for the "elbow" in the plot.

**Steps to find the elbow point:**
1.   Calculate the cumulative explained variance.
2.   Compute the "rate of change" (difference) in the explained variance between successive components.
3.  The elbow point corresponds to the component where the rate of change starts to decrease sharply, indicating the point beyond which adding more components contributes significantly less to the explained variance.

In [ ]:
# Compute the rate of change in explained variance
rate_of_change = np.diff(explained_variance)

# Find the elbow point (the point where the rate of change drops)
thresold = 0.001
elbow_point = np.argmax(rate_of_change < thresold) + 1  # +1 because np.diff reduces the array size by 1

# Create the plot
plt.figure(figsize=(8, 6))
plt.plot(range(1, n_components + 1), explained_variance, marker='o', linestyle='--', color='b')
plt.axvline(elbow_point, color='r', linestyle='--', label=f'Elbow Point: {elbow_point}')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Convergence: Cumulative Explained Variance vs. Number of Components')
plt.grid(True)
plt.legend()
plt.show()

# Print the elbow point
print(f"The elbow point is at component number: {elbow_point}")

**EXERCIZE:**  Train again the machine learning model but with the scaled reuduced data now.

In [ ]:
# Build the Kriging model on the data: define a kernel for the Gaussian Process
initial_kernel = C(1.0, (1e2, 1e5)) * RBF(1.0, (1e-2, 1e2)) + WhiteKernel(noise_level=1e-2)

# Create the GaussianProcessRegressor with the defined kernel
gp = GaussianProcessRegressor(kernel=initial_kernel, n_restarts_optimizer=10, random_state=42)

# Measure the time taken to fit the model
start_time = time.time()
gp.fit( , )
end_time = time.time()

# Compute the computational time
fit_time = end_time - start_time
print(f"Time taken to fit the Gaussian Process model: {fit_time:.4f} seconds")

# Predict using the Kriging model on the test data
y_pred_scaled_reduced = gp.predict()

# Inverse transform to reconstruct the original output
y_pred_scaled = pca_simulations.inverse_transform()
y_pred = scaler_y.inverse_transform()

# Calculate RMSE for each case
rmse_vec = []
for i in range(y_test.shape[0]):
    rmse = np.sqrt(mean_squared_error(y_test[i, :], y_pred[i, :]))
    rmse_vec.append(rmse)

print(f"RMSE: {np.mean(rmse_vec)} mm")

# Compute the relative error based on the mean radius
relative_error = (np.mean(rmse_vec) / average_radius) * 100
print(f"Relative Error: {relative_error:.2f}%")

The computational time is improved drastically while maintaing the precision of the previous case! We can now visualise the results of the prediction to better understand the scale of these errors.

In [ ]:
# Specify the case
N = 100
case = 20  # Case number to use from y_pred

# Specify the name of the position file
position_file = "Input/pos_stent.txt"  # Position data file (positions for the stent)
connectivity_file = "Input/conn_stent.txt"

# Read the position matrix from position_file
position = np.loadtxt(position_file)
connectivity = np.loadtxt(connectivity_file).astype(np.int32)

# Ensure that case is within the range of y_pred (predicted displacements)
if case <= len(y_pred):  # Assuming y_pred is a list of lists or a 2D array where each row is a displacement vector
    displacement_pred_vector = y_pred[case - 1]  # Select the specified case from y_pred (predicted displacement)
else:
    raise ValueError(f"Case {case} does not exist in y_pred.")

# Reshape the displacement vector into a matrix with 3 columns
displacement_pred = np.array(displacement_pred_vector).reshape(-1, 3)

# Add the position matrix and predicted displacement matrix element-wise to get the predicted final position matrix
final_position_pred = position + displacement_pred

# Ensure that case is within the range of y_test (actual displacements)
if case <= len(y_test):  # Assuming y_test is a list of lists or a 2D array where each row is a displacement vector
    displacement_actual_vector = y_test[case - 1]  # Select the specified case from y_test (actual displacement)
else:
    raise ValueError(f"Case {case} does not exist in y_test.")

# Reshape the displacement vector into a matrix with 3 columns
displacement_actual = np.array(displacement_actual_vector).reshape(-1, 3)

# Add the position matrix and actual displacement matrix element-wise to get the final position matrix
final_position_actual = position + displacement_actual

# Create a visualizer object and add both stents to it
visualizer = utils.Visualizer()
visualizer.add_stent(final_position_pred, connectivity, color="green")  # Predicted stent in green
visualizer.add_stent(final_position_actual, connectivity, color="red")  # Actual stent in red

# Update the control point and radius based on parameters from a different file
parameter_file = f"Input/parameters_{N}.txt"

with open(parameter_file, 'r') as parameter_file_handle:
    # Read all lines from the parameter file
    lines = parameter_file_handle.readlines()
    if idx_test[case - 1] <= len(lines):  # Ensure that case is within the number of available lines
        parameter_vector = list(map(float, lines[idx_test[case - 1]].strip().split()))  # Read and convert the line to floats
    else:
        raise ValueError(f"Case {idx_test[case - 1]} does not exist in the parameter file.")

# Define the control point, vessel radius, and deployment site from the parameters
control_point = np.array([parameter_vector[1], 0, parameter_vector[2]], dtype=np.float64)
vessel_radius = parameter_vector[0]
deployment_site = parameter_vector[3]

# Update the vessel display with the new control point and radius
visualizer.add_vessel(control_point, vessel_radius, deployment_site)
visualizer.show()

**EXERCISE:** Write in a single cell all the passages for building a ROM with scaled and reduced data but now using the largest dataset. Visualise the predicted stent and compare it with the one predicted using the machine learning trained with the smallest dataset.

# Extra

We can now compare the performance of the machine learning model when we use a


In [ ]:
# Prepare lists to store the errors
N_values = [100, 250, 500, 1000]
rmse_results = []
rmse_std_results = []

for N in N_values:

    # Example usage
    parameters_file = f"Input/parameters_{N}.txt"  # Replace with your actual file name
    simulations_file = f"Input/simulations_{N}.txt"  # Replace with your actual file name

    # Load the data
    parameters, simulations = load_data(parameters_file, simulations_file)
    indices = np.arange(len(parameters))

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(parameters, simulations, indices, test_size=0.25, random_state=42)

    # Save the idx_test (indices of the test set) to a text file
    idx_test_file = f"Input/idx_test_{N}.txt"
    np.savetxt(idx_test_file, idx_test, fmt='%d', delimiter="\t")

    # Standardize both training and test sets for the output
    scaler_y = StandardScaler()
    y_train_scaled = scaler_y.fit_transform(y_train)
    y_test_scaled = scaler_y.transform(y_test)

    # Standardize both training and test sets for the input
    scaler_x = StandardScaler()
    X_train_scaled = scaler_x.fit_transform(X_train)
    X_test_scaled = scaler_x.transform(X_test)

    # Create the PCA on the output (simulations)
    n_components = 50
    pca_simulations = PCA(n_components=n_components)

    # Fit and transform the output
    y_train_scaled_reduced = pca_simulations.fit_transform(y_train_scaled)

    # Build the Kriging model on the data: define a kernel for the Gaussian Process
    initial_kernel = C(1.0, (1e2, 1e5)) * RBF(1.0, (1e-2, 1e2)) + WhiteKernel(noise_level=1e-2)

    # Create the GaussianProcessRegressor with the defined kernel
    gp = GaussianProcessRegressor(kernel=initial_kernel, n_restarts_optimizer=10, random_state=42)

    # Fit the Gaussian Process model to the reduced data
    gp.fit(X_train_scaled, y_train_scaled_reduced)

    # Predict using the Kriging model on the test data
    y_pred_scaled_reduced = gp.predict(X_test_scaled)

    # Inverse transform to reconstruct the original output
    y_pred_scaled = pca_simulations.inverse_transform(y_pred_scaled_reduced)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)

    # Save predictions to a text file
    prediction_file = f"Input/predictions_{N}.txt"
    np.savetxt(prediction_file, y_pred, fmt='%.6f', delimiter="\t")

    # Calculate RMSE for each case
    rmse_vec = []
    for i in range(y_test.shape[0]):
        rmse = np.sqrt(mean_squared_error(y_test[i, :], y_pred[i, :]))
        rmse_vec.append(rmse)

    print(f"Number of training cases: {N}")
    print(f"  mean of RMSE: {np.mean(rmse_vec)} mm")
    print(f"  median of RMSE: {np.median(rmse_vec)} mm ")
    print(f"  sd of RMSE: {np.std(rmse_vec)} mm")


    # Compute the relative error based on the mean radius
    relative_error = (np.mean(rmse_vec) / average_radius) * 100
    print(f"  relative error: {relative_error:.2f}%")

    # Store the results
    rmse_results.append(np.mean(rmse_vec))
    rmse_std_results.append(np.std(rmse_vec))

# Plotting MAE and RMSE vs. number of training samples
plt.figure(figsize=(10, 6))
plt.errorbar(N_values, rmse_results, yerr=rmse_std_results, label='RMSE', marker='o', capsize=5)
plt.xlabel("Number of Training Samples")
plt.ylabel("Error")
plt.title("Error vs Number of Training Samples")
plt.legend()
plt.grid(True)
plt.show()


Finally, let's visualise the stent predited by training the machine learning model with `N = 1000` cases.

In [ ]:
# Specify the database and case
N = 500
case = 3  # Case number to use from y_pred and y_actual

# Specify the name of the position file
position_file = "Input/pos_stent.txt"  # Position data file (positions for the stent)
connectivity_file = "Input/conn_stent.txt"

# Read the position matrix from position_file
position = np.loadtxt(position_file)
connectivity = np.loadtxt(connectivity_file).astype(np.int32)

# Read the predicted displacements (y_pred) from the prediction file (one line for each case)
predictions_file = f"Input/predictions_{N}.txt"
y_pred = np.loadtxt(predictions_file)

# Ensure that case is within the range of y_pred (predicted displacements)
if case <= len(y_pred):  # Assuming y_pred is a list of rows where each row is a displacement vector
    displacement_pred_vector = y_pred[case - 1]  # Select the specified case (predicted displacement)
else:
    raise ValueError(f"Case {case} does not exist in y_pred.")

# Reshape the displacement vector into a matrix with 3 columns
displacement_pred = np.array(displacement_pred_vector).reshape(-1, 3)

# Add the position matrix and predicted displacement matrix element-wise to get the predicted final position matrix
final_position_pred = position + displacement_pred

# Read the actual displacements (y_actual) from the simulations file (one line for each case)
simulations_file = f"Input/simulations_{N}.txt"
y_actual = np.loadtxt(simulations_file)

# Read idx_test from the previously saved text file
idx_test_file = f"Input/idx_test_{N}.txt"
idx_test = np.loadtxt(idx_test_file, dtype=int)  # Read idx_test values from the file

# Ensure that case is within the range of idx_test (test indices)
if idx_test[case - 1] <= len(y_actual):  # Ensure the case is within the range of actual displacements
    displacement_actual_vector = y_actual[idx_test[case - 1]]  # Select the specified case (actual displacement)
else:
    raise ValueError(f"Case {idx_test[case - 1]} does not exist in y_actual.")

# Reshape the displacement vector into a matrix with 3 columns
displacement_actual = np.array(displacement_actual_vector).reshape(-1, 3)

# Add the position matrix and actual displacement matrix element-wise to get the final position matrix
final_position_actual = position + displacement_actual

# Create a visualizer object and add both stents to it
visualizer = utils.Visualizer()
visualizer.add_stent(final_position_pred, connectivity, color="green")  # Predicted stent in green
visualizer.add_stent(final_position_actual, connectivity, color="red")  # Actual stent in red

# Update the control point and radius based on parameters from a different file
parameter_file = f"Input/parameters_{N}.txt"

with open(parameter_file, 'r') as parameter_file_handle:
    # Read all lines from the parameter file
    lines = parameter_file_handle.readlines()
    if idx_test[case - 1] <= len(lines):  # Ensure that case is within the number of available lines
        parameter_vector = list(map(float, lines[idx_test[case - 1]].strip().split()))  # Read and convert the line to floats
    else:
       raise ValueError(f"Case {idx_test[case - 1]} does not exist in the parameter file.")

# Define the control point, vessel radius, and deployment site from the parameters
control_point = np.array([parameter_vector[1], 0, parameter_vector[2]], dtype=np.float64)
vessel_radius = parameter_vector[0]
deployment_site = parameter_vector[3]

# Update the vessel display with the new control point and radius
visualizer.add_vessel(control_point, vessel_radius, deployment_site)
visualizer.show()

We can now interactively visualize the effect of the vessel parameters not only on the vessel shape itself, but also on how it affects the deployed stent configuration. Notice how fast this prediction is compared to a full finite element simulation.

In [ ]:
# Interactively play with the parameters of the vessel and visualize the predicted stent configuration
viz = utils.Visualizer()
viz.add_vessel()
viz.add_stent_from_file("Input/pos_stent.txt", "Input/conn_stent.txt")

def update_vessel_and_stent(x, z, r, s):
    control_point = np.array([x, 0, z], dtype=np.float64)
    viz.update_vessel(control_point, r, s)
    # Predict deformed stent position
    X_scaled = scaler_x.transform(np.array([[r, x, z, s]]))
    y_pred_scaled_reduced = gp.predict(X_scaled)
    y_pred = pca_simulations.inverse_transform(y_pred_scaled_reduced)
    y_pred = scaler_y.inverse_transform(y_pred)
    displacement_pred = np.array(y_pred[0]).reshape(-1, 3)
    final_position_pred = position + displacement_pred
    viz.update_stent(final_position_pred)

    viz.show()

w = interactive(update_vessel_and_stent, x=(control_point_x_bounds[0], control_point_x_bounds[1], 0.1), z=(control_point_z_bounds[0], control_point_z_bounds[1], 0.1), r=(vessel_radius_bounds[0], vessel_radius_bounds[1], 0.1), s=(deploy_site_bounds[0], deploy_site_bounds[1], 0.01))
for c in w.children:
    c.continuous_update = False
display(w)